# kakao 주소 검색 API를 활용하여 X_Y 좌표 수집

- [좌표X,좌표Y 모두 맵핑](../data/unique_apt_group_xy.csv)
- [아파트 중복제거 좌표X,좌표Y 결측치 있음](../data/unique_apt_group.csv)

In [1]:
# sys.path를 수정하여 모듈 경로를 설정
import sys
import os

# 프로젝트 루트 경로 추가
project_root = os.path.abspath("..")
sys.path.append(project_root)
import module

In [2]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# AppleGothic
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
df = module.file_load.unique_apt_group_file()

In [4]:
df.shape

(9032, 12)

In [6]:
print(df['시군구'].isnull().sum())
print(df['아파트명'].isnull().sum())
print(df['번지'].isnull().sum())
print(df[df['아파트명'] == " "].shape)

0
0
0
(0, 12)


In [7]:
(x,y) = module.api.get_coordinates('서울특별시 구로구 구로동 92')
print(x,y)

126.893453118769 37.4949593550093


In [9]:
none_list = filtered_nan_rows = df[df['좌표X'].isna()]
none_list.shape

(8274, 12)

In [10]:
df.loc[758]

아파트_unique_key    강남구_개포동_189_개포주공4단지_1982
시                                    서울특별시
구                                      강남구
동                                      개포동
시군구                          서울특별시 강남구 개포동
번지                                     189
본번                                   189.0
아파트명                               개포주공4단지
건축년도                                  1982
도로명                                 삼성로 14
좌표Y                                    NaN
좌표X                                    NaN
Name: 758, dtype: object

In [11]:
df['아파트_unique_key'].isnull().sum()

0

In [12]:
# api_cache = {}

# def find_cash_and_save(query):
#     if query in api_cache:
#         return api_cache[query]
#     else:
#         (x,y) = module.api.get_coordinates(query)
#         api_cache[query] = (x,y)
#         return (x,y)

In [13]:
api_cache = pd.read_csv('api_cache.csv')
api_cache = api_cache.set_index('query')
def find_cash_and_save(query):
    if query in api_cache.index:
        return (api_cache.loc[query]['x'], api_cache.loc[query]['y'])
    else:
        (x,y) = module.api.get_coordinates(query)
        api_cache.loc[query] = [x,y]
        return (x,y)


In [12]:
# api_cache['x'] = api_cache['x'].astype(str)
# api_cache['y'] = api_cache['y'].astype(str)

In [ ]:
# for index, row in none_list.iterrows():
for index, row in df.iterrows():
    if pd.isna(row['좌표X']):
        continue
    # try:
    query = module.api.search_apt_query(row)
    (x,y) = find_cash_and_save(query)
    if x != None:
        df.loc[index, '좌표X'] = x
        df.loc[index, '좌표Y'] = y
    else:
        print(index, row['아파트명'], query)
    # except Exception as e:
    #     print(e)
    #     print('err', index, query)
    #     print(row)
    #     break

In [27]:
df['좌표X'].isnull().sum()  
df.to_csv('../data/unique_apt_group_xy_v3.csv', index=False)

In [ ]:
df['좌표X'].isnull().sum()  


In [ ]:
df[df['좌표X'].isnull()]

In [38]:
for index, row in df.iterrows():
    if row['좌표X'] != None:
        continue

    print(row['좌표X'], row['좌표Y'])
    # query = module.api.search_apt_query(row)
    # query = row['시군구'] +  " " + row['아파트명']
    # print(query)
    # (x,y) = module.api.get_coordinates(query)
    # if x != None:
    #     print('찾음',query, x, y)
    #     # df.loc[index, '좌표X'] = x
    #     # df.loc[index, '좌표Y'] = y
    # else:
    #     print(index, row['아파트명'], query)

In [24]:
api_cache.to_csv('api_cache_v2.csv')

In [13]:
# # pd.DataFrame(api_cache).to_csv('api_cache.csv', index=False)
# def save_api_cache_to_csv(api_cache, file_name='api_cache.csv'):
#     # 딕셔너리를 DataFrame으로 변환
#     data = [{'query': query, 'x': coords[0], 'y': coords[1]} for query, coords in api_cache.items()]
#     df = pd.DataFrame(data)
    
#     # CSV 파일로 저장
#     df.to_csv(file_name, index=False, encoding='utf-8')

# # 사용 예시
# save_api_cache_to_csv(api_cache)

In [16]:
df['좌표X'].isnull().sum()

In [15]:
df.to_csv('../data/unique_apt_group_xy_v2.csv', index=False)

In [ ]:
api_cache

In [15]:
# df.to_csv('unique_apt_group_임시.csv', index=False)

In [ ]:
df['좌표X'].isnull().sum()

In [ ]:
df.shape

In [17]:
unique = pd.read_csv('../data/unique_apt_group_xy.csv')

In [ ]:
xy_df = pd.read_csv('../data/x_y_key_map.csv')

In [ ]:
unique[unique['좌표X'].isna()].head(1)

In [ ]:
xy_df.head(1)

## 결측치 보완

In [19]:
결측치_df = df.copy()

In [ ]:
결측치_df['좌표X'].isnull().sum()

In [ ]:
unique.head(1)

In [ ]:
결측치_df.head(1)

In [ ]:
for index, row in unique[unique['좌표X'].isna()].iterrows():
    key_query = row['시군구'] + ' ' + row['도로명']
    try:
        x = xy_df[xy_df['search_key1'] == key_query]['좌표X'].values[0]
        y = xy_df[xy_df['search_key1'] == key_query]['좌표Y'].values[0]
        unique.loc[index, '좌표X'] = x
        unique.loc[index, '좌표Y'] = y
    except:
        print('err', index, row['아파트명'])
        continue

In [29]:
# unique.to_csv('../data/unique_apt_group_xy.csv', index=False)